read

In [3]:
#!/usr/bin/python3
import timeit
import glob, os
from os import listdir
import string
import re
import json
import pandas as pd
from gensim.utils import SaveLoad
from gensim.models.word2vec import Word2Vec
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from itertools import islice
from gensim.models import KeyedVectors
import csv
from tqdm import tqdm
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import wordnet as wn
from nltk.stem import PorterStemmer, WordNetLemmatizer
#from nltk import pos_tag, word_tokenize
from nltk.tag import PerceptronTagger
from nltk.corpus import stopwords  # Import the stop word list

In [4]:
# Pywsd's Lemmatizer.
porter = PorterStemmer()
wnl = WordNetLemmatizer()
tagger = PerceptronTagger()
pos_tag = tagger.tag
tokenizer = RegexpTokenizer(r'\w+')


def lemmatize(ambiguous_word, pos=None, neverstem=True, 
              lemmatizer=wnl, stemmer=porter):
    """
    Tries to convert a surface word into lemma, and if lemmatize word is not in
    wordnet then try and convert surface word into its stem.
    This is to handle the case where users input a surface word as an ambiguous 
    word and the surface word is a not a lemma.
    """
    if pos:
        lemma = lemmatizer.lemmatize(ambiguous_word, pos=pos)
    else:
        lemma = lemmatizer.lemmatize(ambiguous_word)
    stem = stemmer.stem(ambiguous_word)
    # Ensure that ambiguous word is a lemma.
    if not wn.synsets(lemma):
        if neverstem:
            return ambiguous_word
        if not wn.synsets(stem):
            return ambiguous_word
        else:
            return stem
    else:
        return lemma

def penn2morphy(penntag, returnNone=False):
    morphy_tag = {'NN':wn.NOUN, 'JJ':wn.ADJ,
                  'VB':wn.VERB, 'RB':wn.ADV}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return None if returnNone else ''

def word_tokenize(text,tokenize=tokenizer):
    #return tokenize.tokenize(text.lower())# doesn't remove stop words
    return [w for w in tokenize.tokenize(text.lower()) if not w in stopwords.words("english")]

def lemmatize_sentence(sentence, neverstem=False, keepWordPOS=False, 
                       tokenizer=word_tokenize, postagger=pos_tag, 
                       lemmatizer=wnl, stemmer=porter):
    words, lemmas, poss = [], [], []
    for word, pos in postagger(tokenizer(sentence)):
        pos = penn2morphy(pos)
        lemmas.append(lemmatize(word.lower(), pos, neverstem,
                                lemmatizer, stemmer))
        poss.append(pos)
        words.append(word)
    if keepWordPOS:
        return words, lemmas, [None if i == '' else i for i in poss]
    return lemmas

In [5]:
def extract(path, sentence_stream):
    count = 0
    subfolder_list = glob.glob(path +'/*')
    pbar = tqdm(total=len(subfolder_list))
    folders = [x[0] for x in os.walk(path)]
    for x in folders[1:-1]:
        #print(count)
        pbar.set_description('Extracting {}'.format(x))
        pbar.update(1)
        for i in listdir(x):
            count += 1
            with open(x + "/" + i, 'r') as f:
                lines = (line for line in islice(f, 7, None))
                try:
                    for line in lines:
                        line = re.sub("\d+\S\d+", 'xxx', line)#remove numbers
                        line = re.sub("\d+", 'xxx', line)#remove numbers
                        sentence_stream +=[lemmatize_sentence(i) for i in line.strip().split('. ')]
                        #sentence_stream +=[word_tokenize(i) for i in line.strip().split('. ')]
                        '''sentence_stream += [
                            list(filter(None, i.strip().lower().translate(
                                str.maketrans(string.punctuation, ' ' * len(string.punctuation))).split(' '))) for i
                            in line.strip().split('. ')]'''
                        # line=[i.strip().lower().replace('^[{}]'.format(string.punctuation), ' ') for i in
                        #                   line]
                except UnicodeDecodeError as e:
                    print(str(e))
    pbar.close()
        # get rid of the null in list.
        # replace punctuation with ' '
        # stop words

In [6]:
sentence_stream = []
#sentence_stream1 = []
#sentence_stream_our = []
start = timeit.default_timer()
print("="*80)
print('start the reuters')
extract("ReutersNews106521", sentence_stream)
print(len(sentence_stream))

start = timeit.default_timer()
sentence_stream = list(filter(None, sentence_stream))
print('after:{}'.format(len(sentence_stream)))
print(timeit.default_timer() - start)
#with open("sentence.csv", "w") as f:
#    writer = csv.writer(f)
#    writer.writerows(sentence_stream)

  0%|          | 0/2588 [00:00<?, ?it/s]

start the reuters


Extracting ReutersNews106521/20131120:  79%|███████▉  | 2045/2588 [10:31:51<1:35:45, 10.58s/it]         

'utf-8' codec can't decode byte 0x8b in position 1: invalid start byte


Extracting ReutersNews106521/20110303: 100%|██████████| 2588/2588 [13:13:51<00:00, 18.40s/it]  


2404385
after:2330421
0.19168934412300587


Create bi-gram/ tri-gram

In [26]:
"""phrases = Phrases(sentence_stream,min_count=500, threshold=2)
bigram = Phraser(phrases)
# print(list(bigram[sentence_stream]))
print(bigram['u', 's', 'wall', 'st', 'wall', 'street','s','p','500','s','p','xxx'])
##TODO phrase
#bigram.save("big_phrase.pickle")
print('finish phrase time:{}'.format(timeit.default_timer() - start))

print('start trigram')
start = timeit.default_timer()
phrases = Phrases(bigram[sentence_stream],min_count=500, threshold=2)
trigram = Phraser(phrases)
#trigram.save("trig_phrase.pickle")
print(trigram[bigram['u', 's', 'wall', 'st', 'wall', 'street','bank','of','america','s','p','500','s','p','xxx']])
print('finish phrase time:{}'.format(timeit.default_timer() - start))"""

'phrases = Phrases(sentence_stream,min_count=500, threshold=2)\nbigram = Phraser(phrases)\n# print(list(bigram[sentence_stream]))\nprint(bigram[\'u\', \'s\', \'wall\', \'st\', \'wall\', \'street\',\'s\',\'p\',\'500\',\'s\',\'p\',\'xxx\'])\n##TODO phrase\n#bigram.save("big_phrase.pickle")\nprint(\'finish phrase time:{}\'.format(timeit.default_timer() - start))\n\nprint(\'start trigram\')\nstart = timeit.default_timer()\nphrases = Phrases(bigram[sentence_stream],min_count=500, threshold=2)\ntrigram = Phraser(phrases)\n#trigram.save("trig_phrase.pickle")\nprint(trigram[bigram[\'u\', \'s\', \'wall\', \'st\', \'wall\', \'street\',\'bank\',\'of\',\'america\',\'s\',\'p\',\'500\',\'s\',\'p\',\'xxx\']])\nprint(\'finish phrase time:{}\'.format(timeit.default_timer() - start))'

In [29]:
#import cpickle as pickle
print("reading sentence from file")
with open("sentence.csv", "r") as f:
    reader = csv.reader(f)
    sentence_stream=list(reader)

reading sentence from file


In [39]:
bigram = Phrases(sentence_stream, min_count=500, threshold=2)
bigram_phraser = Phraser(bigram)
#print(bigram_phraser)

for sent in sentence_stream:
    tokens_ = bigram_phraser[sent]
    #print(tokens_)

In [40]:
print(bigram_phraser['u', 's', 'wall', 'st', 'wall', 'street','s','p','500','s','p','xxx'])
#bigram_phraser.save("big_phrase.pickle")

['u', 's', 'wall', 'st', 'wall_street', 's', 'p', '500', 's', 'p', 'xxx']


In [41]:
trigram = Phrases(bigram[sentence_stream],min_count=500, threshold=2)
trigram_phraser = Phraser(trigram)
#trigram.save("trig_phrase.pickle")
print(trigram_phraser[bigram_phraser['u', 's', 'wall', 'st', 'wall', 'street','bank','of','america','s','p','500','s','p','xxx']])

['u', 's', 'wall', 'st', 'wall_street', 'bank', 'of', 'america', 's', 'p', '500', 's', 'p_xxx']


In [42]:
#trigram_phraser.save("trig_phrase.pickle")

In [44]:
import gensim
# Build the bigram and trigram models
bigram1 = gensim.models.Phrases(sentence_stream, min_count=5, threshold=100) 
trigram1 = gensim.models.Phrases(bigram1[sentence_stream], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram1)
trigram_mod = gensim.models.phrases.Phraser(trigram1)

In [45]:
list(bigram_mod[sentence_stream])[:10]

[['reuters',
  'japan',
  'disgraced',
  'olympus',
  'corp',
  'xxx',
  'may',
  'sell',
  'asset',
  'help',
  'pay',
  'xxx',
  'billion',
  'debt',
  'plan',
  'aim',
  'keep',
  'support',
  'lender',
  'battle',
  'survive',
  'account',
  'scandal',
  'nikkei',
  'business',
  'daily',
  'say',
  'thursday'],
 ['olympus',
  'bank',
  'creditor',
  'crucial',
  'prospect',
  'come',
  'scandal',
  'give',
  'company',
  'relatively',
  'highly',
  'gear',
  'expect',
  'make',
  'hefty',
  'writedowns',
  'account',
  'put',
  'straight'],
 ['proud',
  'maker',
  'cameras_medical',
  'equipment',
  'put',
  'forward',
  'proposal',
  'meeting',
  'creditor',
  'wednesday',
  'offer',
  'cut',
  'debt',
  'xxx',
  'billion',
  'yen',
  'xxx',
  'billion',
  'next',
  'three',
  'year',
  'nikkei',
  'say'],
 ['appear',
  'consider',
  'sell',
  'asset',
  'mean',
  'repay',
  'debt',
  'addition',
  'tap',
  'cash',
  'reserve',
  'cash',
  'flow',
  'newspaper',
  'say',
  'unsou

In [46]:
print(bigram_mod['u', 's', 'wall', 'st', 'wall', 'street','s','p','500','s','p','xxx'])

['u', 's', 'wall', 'st', 'wall_street', 's', 'p', '500', 's', 'p', 'xxx']
